---
layout: page
title: Comfortable Lifestyle Retirement
---

In [1]:
""" import library and package dependencies """
from sys import path
path.insert(0, '../src')
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from IPython.display import display, Markdown

from chart_format import StandardChart, LogChart, PercentileChart

## How much do I need to retire?

The amount of savings required at retirement age primarily depends upon the lifestyle retirement expenses, and how long the funds will need to cover these expenses. 

ASFA research suggests a comfortable retirement lifestyle needs to cover expenses of around $75,000 per year. The ASFA calculate the expenses for retirees aged 85 and over are approx 7% lower than those aged 65, because over 85 years any increased medical expenses are offset by significant reduction in travel and leisure expenses. See [ASFA Retirement Expenses adjusted for inflation](retirement-expenses.md).

For a simplified model, assume retirement funds are required from retirement age 65 till age 92, when the retirement fund balance will be zero. 

## Government Age Pension

The Age Pension has a range of eligibility requirements that take home ownership and personal assets into consideration. 

The full pension provides around $45,000 for a retired couple if their combined income is below around $10,000 per year and personal assets (excluding home) are below around $500,000.

All income from the government Age Pension will be excluded from this simplified model.

In [2]:
retirement_age = 50
fund_depletion_age = 91
reduced_mobillity_age = 85
reduced_mobility_expense_rate = 7
living_expenses = 75000
inflation_rate = 2.75
investment_return_rate = 6

In [3]:
def create_retirement_amortization_table(retirement_age, fund_depletion_age, reduced_mobility_age, 
                                       living_expenses, reduced_mobility_expense_rate, 
                                       investment_return_rate, inflation_rate):
    """
    Create inflation-adjusted amortization table with investment returns and inflation as deflator
    
    Parameters:
    retirement_age (int): Age when retirement begins
    fund_depletion_age (int): Age when fund balance reaches zero
    reduced_mobility_age (int): Age when expenses reduce due to reduced mobility
    living_expenses (float): Annual living expenses before reduced mobility
    reduced_mobility_expense_rate (float): Percentage reduction in expenses after reduced mobility
    investment_return_rate (float): Annual investment return rate (%)
    inflation_rate (float): Annual inflation rate (%)
    
    Returns:
    pandas.DataFrame:
    """
    # Calculate real investment return rate
    real_return_rate = investment_return_rate - inflation_rate
    
    # Calculate reduced mobility expenses
    reduced_mobility_expenses = living_expenses * (1 - reduced_mobility_expense_rate / 100)
    
    # Create lists to store the inflation-adjusted data
    ages_real = []
    annual_expenses_real = []
    start_balance_real = []
    investment_return_amount = []
    inflation_amount = []
    real_return_amount = []
    end_balance_real = []
    
    # Work backwards from fund depletion age to calculate required balances
    current_balance_real = 0  # Start with zero at the end
    
    # Build the table working backwards, then reverse for display
    for year in range(fund_depletion_age - retirement_age, -1, -1):
        age = retirement_age + year
        
        # Determine expense for this year
        if age >= reduced_mobility_age:
            expense = reduced_mobility_expenses
        else:
            expense = living_expenses
        
        # Calculate required start balance for this year
        required_start_balance = (current_balance_real + expense) / (1 + real_return_rate/100)
        
        # Calculate investment components
        investment_return = required_start_balance * (investment_return_rate / 100)
        inflation_erosion = required_start_balance * (inflation_rate / 100)
        real_return = investment_return - inflation_erosion
        
        # Store values (we'll reverse the lists later)
        ages_real.insert(0, age)
        annual_expenses_real.insert(0, expense)
        start_balance_real.insert(0, required_start_balance)
        investment_return_amount.insert(0, investment_return)
        inflation_amount.insert(0, inflation_erosion)
        real_return_amount.insert(0, real_return)
        
        # Calculate end balance: start + real_return - expense
        end_balance = required_start_balance + real_return - expense
        end_balance_real.insert(0, end_balance)
        
        # Set up for next iteration (previous year)
        current_balance_real = required_start_balance
        
    # Create DataFrame for the inflation-adjusted amortization table
    amortization_real_df = pd.DataFrame({
        'Age': ages_real,
        'Start Balance': start_balance_real,
        'Investment Rate': investment_return_rate,
        'Investment Return': investment_return_amount,
        'Inflation Rate': inflation_rate,
        'Inflation Erosion': inflation_amount,
        'Real Return': real_return_amount,
        'Annual Expenses': annual_expenses_real,
        'End Balance': end_balance_real
    })
    
    return amortization_real_df

In [4]:
def format_amortization_display(df):
    """
    Format an amortization dataframe for display with currency and percentage formatting.
    
    Parameters:
    df (pandas.DataFrame): The amortization dataframe to format
    
    Returns:
    pandas.DataFrame: A formatted copy of the dataframe for display
    """
    # Format dataframe for display (keep original values for calculations)
    display_df = df.copy()
    
    # Currency columns 
    currency_cols = ['Start Balance', 'Investment Return', 'Inflation Erosion', 'Real Return', 'Annual Expenses', 'End Balance']
    for col in currency_cols:
        if col in display_df.columns:
            display_df[col] = display_df[col].apply(lambda x: f"${x:,.0f}")
    
    # Percentage columns
    percentage_cols = ['Investment Rate', 'Inflation Rate']
    for col in percentage_cols:
        if col in display_df.columns:
            display_df[col] = display_df[col].apply(lambda x: f"{x}%")
    
    display(Markdown(f"Required starting fund balance: {display_df['Start Balance'].iloc[0]}"))
    display(Markdown(f"Balance values expressed in 'real terms' adjusted for inflation by deflating (eroding) future values."))
    display(Markdown(f"\nAmortization Table:"))

    display(display_df)
    #return display_df

In [5]:
# Basic model with cash account interest rate matching inflation

investment_return_rate = 2.75

amortization_real_df = create_retirement_amortization_table(
    retirement_age, fund_depletion_age, reduced_mobillity_age, 
    living_expenses, reduced_mobility_expense_rate, 
    investment_return_rate, inflation_rate
)

display(Markdown(f"""## Cash Portfolio
Assume the retirement funds are held in a cash account with an interest rate matching inflation, resulting in an annual net return of zero.
\nIn this simplified scenario a fund of ${living_expenses*10:,} will only cover annual expenses of ${living_expenses:,} for ten years.
"""))

display(Markdown(f"""Assumptions:
- Living Expenses: ${living_expenses:,}
- Inflation Rate: {inflation_rate}%
- Investment Return Rate: {investment_return_rate}%
- Fund Depletion Age: {fund_depletion_age}
- Reduced Mobility Age: {reduced_mobillity_age}
- Reduced Mobility Expense Rate: {reduced_mobility_expense_rate}%
"""))

format_amortization_display(amortization_real_df)

## Cash Portfolio
Assume the retirement funds are held in a cash account with an interest rate matching inflation, resulting in an annual net return of zero.

In this simplified scenario a fund of $750,000 will only cover annual expenses of $75,000 for ten years.


Assumptions:
- Living Expenses: $75,000
- Inflation Rate: 2.75%
- Investment Return Rate: 2.75%
- Fund Depletion Age: 91
- Reduced Mobility Age: 85
- Reduced Mobility Expense Rate: 7%


Required starting fund balance: $3,113,250

Balance values expressed in 'real terms' adjusted for inflation by deflating (eroding) future values.


Amortization Table:

,Age,Start Balance,Investment Rate,Investment Return,Inflation Rate,Inflation Erosion,Real Return,Annual Expenses,End Balance
0,50,"$3,113,250",2.75%,"$85,614",2.75%,"$85,614",$0,"$75,000","$3,038,250"
1,51,"$3,038,250",2.75%,"$83,552",2.75%,"$83,552",$0,"$75,000","$2,963,250"
2,52,"$2,963,250",2.75%,"$81,489",2.75%,"$81,489",$0,"$75,000","$2,888,250"
3,53,"$2,888,250",2.75%,"$79,427",2.75%,"$79,427",$0,"$75,000","$2,813,250"
4,54,"$2,813,250",2.75%,"$77,364",2.75%,"$77,364",$0,"$75,000","$2,738,250"
5,55,"$2,738,250",2.75%,"$75,302",2.75%,"$75,302",$0,"$75,000","$2,663,250"
6,56,"$2,663,250",2.75%,"$73,239",2.75%,"$73,239",$0,"$75,000","$2,588,250"
7,57,"$2,588,250",2.75%,"$71,177",2.75%,"$71,177",$0,"$75,000","$2,513,250"
8,58,"$2,513,250",2.75%,"$69,114",2.75%,"$69,114",$0,"$75,000","$2,438,250"
9,59,"$2,438,250",2.75%,"$67,052",2.75%,"$67,052",$0,"$75,000","$2,363,250"


In [6]:
investment_return_rate = 6

amortization_real_df = create_retirement_amortization_table(
    retirement_age, fund_depletion_age, reduced_mobillity_age, 
    living_expenses, reduced_mobility_expense_rate, 
    investment_return_rate, inflation_rate
)

display(Markdown(f"""## Balanced Portfolio
Assume the retirement funds are held in a balanced or defensive portfolio with an annual net return (after fees and taxes) of {investment_return_rate}%.
"""))

format_amortization_display(amortization_real_df)

## Balanced Portfolio
Assume the retirement funds are held in a balanced or defensive portfolio with an annual net return (after fees and taxes) of 6%.


Required starting fund balance: $1,694,838

Balance values expressed in 'real terms' adjusted for inflation by deflating (eroding) future values.


Amortization Table:

,Age,Start Balance,Investment Rate,Investment Return,Inflation Rate,Inflation Erosion,Real Return,Annual Expenses,End Balance
0,50,"$1,694,838",6%,"$101,690",2.75%,"$46,608","$55,082","$75,000","$1,674,920"
1,51,"$1,674,920",6%,"$100,495",2.75%,"$46,060","$54,435","$75,000","$1,654,355"
2,52,"$1,654,355",6%,"$99,261",2.75%,"$45,495","$53,767","$75,000","$1,633,122"
3,53,"$1,633,122",6%,"$97,987",2.75%,"$44,911","$53,076","$75,000","$1,611,198"
4,54,"$1,611,198",6%,"$96,672",2.75%,"$44,308","$52,364","$75,000","$1,588,562"
5,55,"$1,588,562",6%,"$95,314",2.75%,"$43,685","$51,628","$75,000","$1,565,190"
6,56,"$1,565,190",6%,"$93,911",2.75%,"$43,043","$50,869","$75,000","$1,541,059"
7,57,"$1,541,059",6%,"$92,464",2.75%,"$42,379","$50,084","$75,000","$1,516,143"
8,58,"$1,516,143",6%,"$90,969",2.75%,"$41,694","$49,275","$75,000","$1,490,418"
9,59,"$1,490,418",6%,"$89,425",2.75%,"$40,986","$48,439","$75,000","$1,463,856"


## Comparison

Many retirement plans use similar assumptions for retirement funds with higher investment options for wealth accumulation prior to retirement.

Both the ASFA Retirement Standard and the moneysmart.gov.au calculator include a part Age Pension in their calculations, and the required funds are **reduced by half**. 

Using the same assumptions of "2.75% AWE as a deflator, and investment earning of 6%" 

The ASFA March 2025 update suggests a couple require a fund balance of $690,000 at age 67 in todays dollars, but is based on slightly lower expenses of $73,000 and does not disclose the fund depletion age or how long the funds would be expected to last. 

The moneysmart.gov.au calculator suggests a couple require a only $450,000 at age 67 with the part Age Pension starting from age 67 at $45,246 and slowly ramping over $56,000 by age 91.

SuperGuy Youtube channel also suggest a comfortable retirement will require around $700,000 age 67 based on different assumptions of higher annual expenses of $80,000, with higher returns of 6.5% but lower inflation at 2.5% and fund depletion at age 90 instead of age 92.
https://www.youtube.com/watch?v=zT32JNvI17U

SuperGuy demonstrated how much difference the Age Pension can make with a couple retiring at age 60 require double the fund balance, around $1.45 million.

The Age Pension thresholds are based on asset / home ownership, so non-homeowner claim higher Age Pension, but would need higher expenses to cover rent payments.

In [7]:
investment_return_rate = 12

amortization_real_df = create_retirement_amortization_table(
    retirement_age, fund_depletion_age, reduced_mobillity_age, 
    living_expenses, reduced_mobility_expense_rate, 
    investment_return_rate, inflation_rate
)

display(Markdown(f"""## Growth Portfolio
Assume the retirement funds are held in a growth portfolio tied to the S&P 500 index with an annual net return (with dividends and stock buybacks, less fees and taxes) of {investment_return_rate}%.
"""))

format_amortization_display(amortization_real_df)

## Growth Portfolio
Assume the retirement funds are held in a growth portfolio tied to the S&P 500 index with an annual net return (with dividends and stock buybacks, less fees and taxes) of 12%.


Required starting fund balance: $789,892

Balance values expressed in 'real terms' adjusted for inflation by deflating (eroding) future values.


Amortization Table:

,Age,Start Balance,Investment Rate,Investment Return,Inflation Rate,Inflation Erosion,Real Return,Annual Expenses,End Balance
0,50,"$789,892",12%,"$94,787",2.75%,"$21,722","$73,065","$75,000","$787,957"
1,51,"$787,957",12%,"$94,555",2.75%,"$21,669","$72,886","$75,000","$785,843"
2,52,"$785,843",12%,"$94,301",2.75%,"$21,611","$72,690","$75,000","$783,533"
3,53,"$783,533",12%,"$94,024",2.75%,"$21,547","$72,477","$75,000","$781,010"
4,54,"$781,010",12%,"$93,721",2.75%,"$21,478","$72,243","$75,000","$778,254"
5,55,"$778,254",12%,"$93,390",2.75%,"$21,402","$71,988","$75,000","$775,242"
6,56,"$775,242",12%,"$93,029",2.75%,"$21,319","$71,710","$75,000","$771,952"
7,57,"$771,952",12%,"$92,634",2.75%,"$21,229","$71,406","$75,000","$768,358"
8,58,"$768,358",12%,"$92,203",2.75%,"$21,130","$71,073","$75,000","$764,431"
9,59,"$764,431",12%,"$91,732",2.75%,"$21,022","$70,710","$75,000","$760,141"


In [8]:
investment_return_rate = 30

amortization_real_df = create_retirement_amortization_table(
    retirement_age, fund_depletion_age, reduced_mobillity_age, 
    living_expenses, reduced_mobility_expense_rate, 
    investment_return_rate, inflation_rate
)

display(Markdown(f"""## Exponential Growth Portfolio
Assume the retirement funds are held in an exponential growth portfolio with an annual net return (with dividends and stock buybacks, less fees and taxes) of {investment_return_rate}%.
"""))

format_amortization_display(amortization_real_df)

## Exponential Growth Portfolio
Assume the retirement funds are held in an exponential growth portfolio with an annual net return (with dividends and stock buybacks, less fees and taxes) of 30%.


Required starting fund balance: $275,215

Balance values expressed in 'real terms' adjusted for inflation by deflating (eroding) future values.


Amortization Table:

,Age,Start Balance,Investment Rate,Investment Return,Inflation Rate,Inflation Erosion,Real Return,Annual Expenses,End Balance
0,50,"$275,215",30%,"$82,564",2.75%,"$7,568","$74,996","$75,000","$275,211"
1,51,"$275,211",30%,"$82,563",2.75%,"$7,568","$74,995","$75,000","$275,206"
2,52,"$275,206",30%,"$82,562",2.75%,"$7,568","$74,994","$75,000","$275,200"
3,53,"$275,200",30%,"$82,560",2.75%,"$7,568","$74,992","$75,000","$275,191"
4,54,"$275,191",30%,"$82,557",2.75%,"$7,568","$74,990","$75,000","$275,181"
5,55,"$275,181",30%,"$82,554",2.75%,"$7,567","$74,987","$75,000","$275,168"
6,56,"$275,168",30%,"$82,550",2.75%,"$7,567","$74,983","$75,000","$275,151"
7,57,"$275,151",30%,"$82,545",2.75%,"$7,567","$74,979","$75,000","$275,130"
8,58,"$275,130",30%,"$82,539",2.75%,"$7,566","$74,973","$75,000","$275,103"
9,59,"$275,103",30%,"$82,531",2.75%,"$7,565","$74,965","$75,000","$275,068"
